**Check the GPU**

In [ ]:
!nvidia-smi

**Install BLEURT**

In [ ]:
!git clone https://github.com/google-research/bleurt.git
!pip install -q ./bleurt

# !wget https://storage.googleapis.com/bleurt-oss-21/BLEURT-20.zip .
# !unzip BLEURT-20.zip -d "/content/drive/MyDrive/Colab Notebooks/magistrska/models/bleurt-checkpoints"

# !rm BLEURT-20.zip

!pip install -q rouge-score

**Load data to DataFrame**

In [ ]:
import os
import pandas as pd

DIR_PATH = "/content/drive/MyDrive/Colab Notebooks/magistrska/data"

def load_df_from_drive():
  return pd.read_csv(f"{DIR_PATH}/dataframe-kas.csv", encoding="utf-8")

In [ ]:
print("Loading data from drive...")
df = load_df_from_drive()
df = df.dropna()
df = df.reset_index(drop=True)
print("Finished.")

**Split into train, test and validation dataset (80 : 10 : 10)**

In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_and_validation_df = train_test_split(df, train_size=0.8, shuffle=True, random_state=42)
test_df, validation_df = train_test_split(test_and_validation_df, test_size=0.5, shuffle=True, random_state=42)
train_df.shape, test_df.shape, validation_df.shape

del df

**Draw a plot of validation loss change for a model**

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

RESULTS_PATH = "/content/drive/MyDrive/Colab Notebooks/magistrska/results"
testing_filename_csv = f"{RESULTS_PATH}/google/pegasus-x-base/to_eng/percentage_20.0/val_results.csv"
results = pd.read_csv(testing_filename_csv)

loss_values = [float(x.replace(",", ".")) for x in list(results.get("loss"))]
epochs = range(0, len(loss_values))

# Find the epoch with the minimum loss value
min_loss = min(loss_values)
min_epoch = loss_values.index(min_loss)

# Create the plot
plt.figure(figsize=(10, 6))

# Plot the loss values
plt.plot(epochs, loss_values, label='Validacijska izguba', color='blue', linewidth=1)

# Mark the minimum loss value
plt.scatter(min_epoch, min_loss, color='red', zorder=5)
plt.text(min_epoch, min_loss, f'  Min. izguba\n  {min_loss:.3f}', fontsize=12, verticalalignment='bottom', color='red')

# Add title and labels
plt.title(f"Sprememba izgube na validacijski množici", fontsize=16)
plt.xlabel('Iteracija učenja', fontsize=14)
plt.ylabel('Izguba', fontsize=14)

# Add a legend
plt.legend(loc='upper right', fontsize=12)

# Add grid
plt.grid(True, which='both', linestyle='--', linewidth=0.5)

# Save the plot to a file
plt.savefig(f"{RESULTS_PATH}/google/pegasus-x-base/to_eng/percentage_20.0/validation_loss_plot_with_min.png")

# Show the plot
plt.show()

**Load results and get metrics**

In [ ]:
import pandas as pd

PERCENTAGE = 0.2
MODEL_NAME = "long-t5-tglobal-base"
# MODEL_NAME = "pegasus-x-base"
# MODEL_NAME = "bigbird-pegasus-large-arxiv"

RESULTS_PATH = "/content/drive/MyDrive/Colab Notebooks/magistrska/results"
MODEL_RESULTS_PATH = f"{RESULTS_PATH}/google/{MODEL_NAME}/to_eng/percentage_{PERCENTAGE * 100}/testing_results.csv"

results = pd.read_csv(MODEL_RESULTS_PATH)
results.head()

**Export 30 random examples to a JSON file**

In [ ]:
import json
import pandas as pd

random_examples = [6, 12, 47, 631, 56, 134, 219, 326, 403, 639, 183, 272,
                   118, 649, 313, 497, 231, 352, 278, 528, 185, 97, 70, 688,
                   332, 263, 408, 644, 31, 544]

RESULTS_PATH = "/content/drive/MyDrive/Colab Notebooks/magistrska/results"
LONGT5_PATH = f"{RESULTS_PATH}/google/long-t5-tglobal-base/to_eng/percentage_100.0/testing_results.csv"
PEGASUS_X_PATH = f"{RESULTS_PATH}/google/pegasus-x-base/to_eng/percentage_100.0/testing_results.csv"
BIG_BIRD_PATH = f"{RESULTS_PATH}/google/bigbird-pegasus-large-arxiv/to_eng/percentage_100.0/testing_results.csv"

longt5_results = pd.read_csv(LONGT5_PATH)
pegasus_x_results = pd.read_csv(PEGASUS_X_PATH)
big_bird_results = pd.read_csv(BIG_BIRD_PATH)

all_data = {}
for example_id in random_examples:
  example_data = {}
  longt5_row = longt5_results.iloc[example_id]
  pegasus_x_row = pegasus_x_results.iloc[example_id]
  big_bird_row = big_bird_results.iloc[example_id]

  example_data["reference_summary_eng"] = longt5_row["abstract_eng"].replace("\n", "")
  example_data["longt5_generated_summary_eng"] = longt5_row["generated_summary_eng"]
  example_data["pegasus_x_generated_summary_eng"] = pegasus_x_row["generated_summary_eng"]
  example_data["big_bird_generated_summary_eng"] = big_bird_row["generated_summary_eng"]

  all_data[example_id] = example_data

json_data = json.dumps(all_data, indent=4, ensure_ascii=False)

with open(f"{RESULTS_PATH}/analysis-ready.json", "w", encoding='utf-8') as outfile:
  outfile.write(json_data)


**Create 5 text files (first three from above) for ChatGPT summary generation**

In [ ]:
RESULTS_PATH = "/content/drive/MyDrive/Colab Notebooks/magistrska/results"

random_examples = [6, 12, 47, 631, 56]
for example_id in random_examples:
  example = test_df.iloc[example_id]
  example_text = example["text"]

  with open(f"{RESULTS_PATH}/text_{example_id}.txt", "w", encoding='utf-8') as text_file:
    print(f"Generating text_{example_id}.txt in {RESULTS_PATH}...")
    text_file.write(example_text)
    print(f"Done\n")


**Copy results to public results**

In [ ]:
from distutils.dir_util import copy_tree

RESULTS_PATH = "/content/drive/MyDrive/Colab Notebooks/magistrska/results/google/"
PUBLIC_RESULTS_PATH = "/content/drive/MyDrive/Colab Notebooks/magistrska/public results/"

copy_tree(RESULTS_PATH, PUBLIC_RESULTS_PATH, update=True)

**Calculate metrics for ChatGPT-4o summary**

In [ ]:
from bleurt import score
from rouge_score import rouge_scorer

bleurt = score.BleurtScorer("drive/MyDrive/Colab Notebooks/magistrska/models/bleurt-checkpoints/BLEURT-20")
rouge = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=True)

reference_summaries = [
    "In the era of the information society is the World Wide Web became one of the most important sources of information, but users in the crowd always harder encountered. There was a trend of building Web portals, which are not only a normal Web page, but the user offer a lot more. They represent the single access point for users to find useful information. Roughly dividing Web portals to the general user portals and business portals. General user portals serve as a gateway for users when they go on the internet, while bu- siness portals combine the information inside an intranet organization and selected links to external Web sites. That will be the Web portal can serve its purpose and satisfy even the most demanding user, it is necessary to proceed very prudently Web project. Subscriber portal must first define the needs and objectives, select the best artist that will satisfy the desires and needs, which will be later transferred to the World Wide Web. Electronic commerce is a form of business where the transactions are made through electronic ne- tworks, mainly the Internet. This is a new way of doing business that does not use paper, but uses modern information and communication technology. There are various areas of electronic commerce. One of them is the elec- tronic banking, which has experienced great development and recent years. Electronic banking is a modern form of business between the bank and cli- ents, which is becoming more common. There are various forms of electronic commerce, such as operating through ATMs, mobile banking, phone banking and internet banking or electronic banking. The greatest expansion of the reach of electronic banking just as the use of the Internet and the population boomed. Like every thing has a certain electronic banking pros and cons, both for the banks as of the end user. The Bank strives to making itself the security of electronic banking has become a leader and developing the business over the internet, sinc ...",
    "Theoretical background: It is a known fact that obesity brings health risks and causes negative consequences. In order to avoid these consequences, we must first identify the causes which are responsible for the development of obesity. One of the main causes is undoubtedly an unhealthy lifestyle which is a result of poor eating habits and insufficient exercise. Aim: The aim of this research was to investigate the adolescent dietary behaviours and physical activity, to determine how the overweight adolescents see their body image and self-esteem and to indentify their knowledge of eating disorders. Method: The research is based on the quantitative descriptive and non-experimental causal method of the empirical research. The instrument for data collection was a written survey questionnaire. We used a non-randomized, purposive sample. The survey was conducted on a sample of 167 male and female students in the ratio of 40:60, in the following secondary schools: Jesenice Secondary School, Secondary School of Nursing Ljubljana and Secondary Engineering School Ljubljana. The data were analyzed with the statistical analysis package SPSS 20. The realization of the sample was 70 per cent. Results: The results of the research showed that (52.4%) of the respondents were satisfied with their body weight, their average age was 18 years. (37.8%) of the students were physically active more than three times a week. (32.3%) of the respondents answered that their weight strongly affected their opinions of themselves. (64.0%) of the students had knowledge of the terms anorexia and bulimia. The first source of information regarding eating disorders was a general practitioner (73.4%). The survey questionnaire showed that the eating habits of adolescents were rather poor. Also, their physical activity proved to be rather unsatisfactory. Discussion: In adolescents, it was found that their eating habits were quite poor, especially the omission of important meals. Their physical activity was rather low (inactive) as compared to the previous generations. However, the adolescents were much better informed about eating disorders (terms anorexia and bulimia), but the familiarity with the treatment of eating disorders was still low. It is important to provide continued health education from the primary school onward, with an emphasis on healthy eating habits throughout life. Clearly, we need further research and analysis of the nutritional patterns of young people, which could provide more insight into their dietary lives.",
    "The study shows various types of lawns. The focus is on the importance of turf as the basis of most of the landscape scheme. The lawn is the basic plot, where the different elements create volume in space. To achieve a quality lawn, we need to know the right steps of development and maintenance of lawn areas. The study is describing types of grass, suitable for lawns in our ecological conditions. Grass species are arranged in a table, according to habitat conditions (light, soil, moisture, temperature), which enables easier selection of grass species for different areas. Lawns can be divided into different types of use - decorative, useful and sport lawns, lawns in the landscape. Lawn is made of different grass species, depending on the type of lawn and site conditions. Preparation and mainteince are very importat. For the turf to serve its purpose, there procedures are various, depending on the type of lawn you maintain. Lawn maintenance consists of a series of measures such as mowing, irrigation, fertilization, regeneration. They are described in individual chapters. Ornamental and sports turf maintenance is more demanding than for general lawns. The study also presents alternatives to traditional lawns that are less demanding for maintenance and more acceptable for the environment (natural - flower meadow, perennials). It also describes the species which are suitable for non - grass lawns. The study presents various principles of grass design. Analysis of the state of public green areas in the city of Kranj showes that a lot of lawn design is unattractive. People don`t use them often, there for they are losing their value.",
    "This research project speaks about improvement of existing warehousing operation in the company TRIFIX Tržič. Intention of this improvement are lower costs of operation, to reduce the stock in the warehouse and to improve efficiency of the warehouse operation with initiation of paper-less operation. The stock in the warehouse makes possible that production and sales works undisturbed, but at the same time it bounds a lot of funds and is the cause of high operating costs. Because of changeable external and internal conditions is the stock sometimes smaller, sometimes bigger. As a lot of factors are changing all the time, also the stock in the warehouse must be changed all the time. The improvement is based on use of new technologies, which enables the upgrade of an existing system, such as electronic capture of data based on the EAN code and electronic buisness operation. The improvement can be made at the same time with the start of use of the EAN code system, execution of analytic method for optimizing of the warehousing, which means definition of signalling stock and optimal order quantities. The improvement of warehouse operation is presented in three steps. First step is with help of analytic method to sort the stocked materials according to the height of the warehousing costs, which are caused by these materials and the definition of normatives of the stocked materials, concerning the highest costs of warehouse operating costs. Second step is the introduction of the EAN code, which represents a base for electronic capture of data. Third step is upgrading of the existing electronic buisness system with MIT engineering software, which enables the introduction of full electronic buisness operation. The improvement is compared to the previos kind of buisness operation system of the company. The improvement is also compared with the old system from the financial point of view.",
    "This diploma present the system INFRANET, the building blocks of different options and links for reliable transmission of alarm messages. Define security requirements for the continuous development and improvement of systems of protection in the technical field. These systems must provide security in the transmission of alarm messages. The security of property from burglary, natural disasters or damage of property due to different impacts depends on appropriate alarm systems. INFRANET is designed as a client-server system with high modularity, decentralized control and optimal utilization of the network. The future of the sistem is in the integration in all relevant communications infrastructure, such as analog subscriber lines, leased lines, ISDN access and X25 networks."
]

chat_gpt_generated_summaries = [
    "The text provided appears to be part of a thesis focusing on the development and implications of electronic banking (e-banking) and web portals, using the SKB bank's online platform as a case study. The thesis emphasizes the growing importance of the internet as a primary source of information and the role that well-designed web portals play in representing companies online. It discusses the key components of e-banking, including various online services, the evolution of banking technologies, and their impact on both businesses and individual users. The research aims to provide a comprehensive overview of e-banking, outline the advantages and disadvantages of web portals, and assess the influence of these portals on user decision-making. The case study on SKB bank is intended to illustrate the practical application of these concepts, highlighting the effectiveness of the bank's online services and the challenges posed by digitalization in the financial sector. The study uses various research methods, including descriptive, comparative, historical, and compilation methods, to explore these issues. Overall, the thesis argues that as e-banking becomes increasingly integral to the banking industry, the design and functionality of web portals are critical to a company's success. Poorly designed portals can lead to significant business losses, while effective ones can enhance customer satisfaction and business performance.",
    "The document discusses the growing issue of obesity among children and adolescents, particularly in developed countries. It emphasizes the significant increase in obesity rates over the last few decades, largely due to lifestyle changes such as reduced physical activity and increased consumption of energy-dense foods. The World Health Organization (WHO) recognizes obesity as a major health challenge of the 21st century, with substantial impacts on public health systems and an increase in related chronic diseases like diabetes, cardiovascular conditions, and even cancer. The text highlights that obesity is not just a physical health issue but also affects emotional well-being, leading to disorders such as bulimia nervosa and binge eating, especially among adolescents. The importance of balanced nutrition during adolescence—a critical period of growth and development—is emphasized, noting that unhealthy eating habits established during this time can lead to lifelong health problems. Furthermore, the document outlines the need for a comprehensive approach to addressing obesity, including proper nutritional assessment, physical activity, and psychological support. It also mentions the role of education in promoting healthy eating habits and the challenges posed by modern lifestyles, such as the prevalence of fast food, sedentary behavior, and the influence of media. The overall message is a call to action for more effective public health strategies to combat the rising trend of obesity in young people.",
    "The thesis explores the significance and management of grassed surfaces, particularly lawns, in urban and landscape architecture. Grassed areas are fundamental to various green spaces, serving both functional and aesthetic roles. The study examines different types of grasses suitable for Slovenia's ecological conditions, detailing their botanical characteristics and environmental requirements. It also discusses the technology involved in the creation and maintenance of lawns, highlighting the challenges such as high costs and environmental impact. As an alternative, natural flowering meadows, which are more ecologically friendly and support biodiversity, are proposed. The thesis further analyzes the design principles of grassed surfaces, including cultivated lawns, natural meadows, and ornamental grasses. It points out that in Slovenia, grassed surfaces often lack creativity and are underutilized, which diminishes their value and leads to neglect. The research includes a case study of grassed areas in Kranj, Slovenia, evaluating their design, use, and maintenance. The findings suggest that by adopting more diverse and ecologically sensitive design approaches, grassed areas can be made more attractive and functional, contributing to urban biodiversity and reducing maintenance costs. The study underscores the need for greater attention to the design and maintenance of grassed surfaces in urban planning to enhance their ecological and aesthetic value.",
    "The thesis focuses on the challenges faced by TRIFIX d.o.o., a Slovenian company specializing in fastening products for construction. The company operates in a rapidly changing environment characterized by technological advancements, economic fluctuations, and increased competition. The author identifies inefficiencies in the company's warehouse and procurement operations, particularly in stock replenishment methods, which are currently based on the warehouse manager's intuition rather than systematic data analysis. This approach leads to increased costs and inefficiencies. The thesis aims to justify the introduction of improvements in the warehouse and procurement processes at TRIFIX d.o.o. The proposed improvements include the implementation of new technologies, such as barcode systems and electronic data capture, to optimize inventory management. Additionally, the author suggests adopting analytical methods to determine safety stock levels and optimal order quantities, which could reduce costs and enhance the company's competitiveness. The thesis also outlines the broader goals of optimizing procurement, storage, and inventory management to reduce overall organizational costs and improve economic efficiency. By focusing on modernizing these processes, the author believes TRIFIX d.o.o. can better navigate the challenges of the global market and maintain its competitive edge. The study includes a detailed analysis of the company's current state, the causes of high costs, and a theoretical exploration of logistics and inventory optimization.",
    "The document provides an in-depth analysis of the state of alarm message transmission systems in Slovenia, particularly focusing on the transition from older systems like Mini TUS to more modern and reliable systems such as INFRANET. The text discusses the limitations of existing alarm transmission methods that rely on traditional telecommunication networks, which may not provide sufficient security. It also explores the potential of integrating the INFRANET system into current telecommunication infrastructures, highlighting its advantages in terms of reliability and its ability to support various alarm-triggered responses, such as in cases of burglary, fire, or technical failures. The INFRANET system, which has been increasingly implemented since October 2008 by Telekom Slovenije, d.d., offers a higher level of reliability in transmitting alarm signals through a specialized network infrastructure. The system allows for remote monitoring and control of various devices, enhancing the efficiency and safety of critical operations such as public transportation and financial services. The document further examines the technical architecture of INFRANET, detailing its backbone and access networks, the integration of customer devices, and the management of network elements via centralized control centers. Moreover, the text provides a comparative analysis of the INFRANET system with its predecessors, such as Mini TUS and ULTRANET, demonstrating the evolution of alarm transmission technologies in Slovenia. It concludes by discussing the future prospects for expanding and enhancing the INFRANET system to meet evolving security needs."
]

bleurt_scores = bleurt.score(
        references=reference_summaries,
        candidates=chat_gpt_generated_summaries
      )

rouge_scores_fmeasure = []
rouge_scores_precision = []
rouge_scores_recall = []
for i in range(len(chat_gpt_generated_summaries)):
  rouge_sc = rouge.score(chat_gpt_generated_summaries[i], reference_summaries[i])["rougeL"]

  rouge_scores_fmeasure.append(rouge_sc.fmeasure)
  rouge_scores_precision.append(rouge_sc.precision)
  rouge_scores_recall.append(rouge_sc.recall)

print(bleurt_scores)
print(rouge_scores_fmeasure)
print(rouge_scores_precision)
print(rouge_scores_recall)